In [1]:
import json
import pandas as pd

In [2]:
contrataciones = json.load(open("contratacionesabiertas_bulk.json", "rb"))

In [3]:
len(contrataciones)

52935

In [4]:
contrataciones[0].keys()

dict_keys(['_id', 'uri', 'version', 'extensions', 'license', 'publishedDate', 'publisher', 'records'])

In [5]:
contrataciones[0]['records'][0].keys()

dict_keys(['ocid', 'releases', 'compiledRelease'])

In [6]:
contrataciones[0]['records'][0]['compiledRelease'].keys()

dict_keys(['tag', 'ciclo', 'date', 'id', 'initiationType', 'language', 'ocid', 'publisher', 'tender', 'contracts', 'parties', 'planning'])

In [7]:
part = contrataciones[0]['records'][0]['compiledRelease']['parties'][0]

In [14]:
contrataciones[0]['records'][0]['compiledRelease']['contracts'][0]["valueWithTax"]['amount']

10387.8

## Buyers/Suppliers

In [44]:
contract = {}
i = 0
for element in contrataciones:
    buyers = []
    suppliers = []
    contract[i] = {}
    contract_value = 0
    for part in element['records'][0]['compiledRelease']['parties']:
        if 'buyer' in part['roles']:
            buyers.append(part['name'])
        elif 'supplier' in part['roles']:
            suppliers.append(part['name'])
        else:
            pass
    try:
        for contrato in element['records'][0]['compiledRelease']['contracts']:
            try:
                contract_value += contrato["valueWithTax"]['amount']
            except:
                contract_value += 0
    except:
        contract_value = 0
    contract[i]['buyers'] = buyers
    contract[i]['suppliers'] = suppliers
    contract[i]['contract_value'] = contract_value
    i += 1

In [45]:
df = pd.DataFrame.from_dict(contract).T

In [47]:
df.head()

,buyers,contract_value,suppliers
0,[delegación SEDESOL en Tabasco],10387.8,[JESUS ALBERTO ROSAS GARDUZA]
1,[delegación SEDESOL en México],739388,[SECRETARIA DE DESARROLLO SOCIAL DELEGACION ES...
2,[delegación SEDESOL en Tabasco],1072,[SECRETARIA DE DESARROLLO SOCIAL DELEGACION TA...
3,[delegación SEDESOL en Tabasco],109620,[SERVISEG S.A. DE C.V.]
4,[delegación SEDESOL en Tabasco],1109.9,[COMERCIALIZADORA COMPUTEL DEL SURESTE S.A. D...


In [48]:
df['len_buyers'] = df["buyers"].apply(len)
df['len_suppliers'] = df["suppliers"].apply(len)

In [49]:
len(df)

52935

In [50]:
df = df[(df['len_buyers'] == 1) & (df["len_suppliers"] == 1)]

In [51]:
del df['len_buyers']
del df["len_suppliers"]

In [52]:
df['buyers'] = df['buyers'].apply(lambda x: x[0])
df['suppliers'] = df['suppliers'].apply(lambda x: x[0])

In [53]:
df.head()

,buyers,contract_value,suppliers
0,delegación SEDESOL en Tabasco,10387.8,JESUS ALBERTO ROSAS GARDUZA
1,delegación SEDESOL en México,739388,SECRETARIA DE DESARROLLO SOCIAL DELEGACION EST...
2,delegación SEDESOL en Tabasco,1072,SECRETARIA DE DESARROLLO SOCIAL DELEGACION TAB...
3,delegación SEDESOL en Tabasco,109620,SERVISEG S.A. DE C.V.
4,delegación SEDESOL en Tabasco,1109.9,COMERCIALIZADORA COMPUTEL DEL SURESTE S.A. DE...


In [55]:
df.to_csv('buyers_suppliers.csv')

In [57]:
text_file = open("contribuyentes_inexistentes.txt", "r")
contribuyentes_inexistentes = text_file.read().split('\n')

In [59]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [60]:
most_similar = []
similarity = []
for index, row in df.iterrows():
    temp = {}
    for contribuyente in contribuyentes_inexistentes:
        temp[contribuyente] = similar(contribuyente, row["suppliers"])
    most_similar.append(max(temp, key=temp.get))
    similarity.append(temp[max(temp)])

In [61]:
df["most_similar"] = most_similar
df["similarity"] = similarity

In [76]:
df.head()

,buyers,contract_value,suppliers,most_similar,similarity
0,delegación SEDESOL en Tabasco,10387.8,JESUS ALBERTO ROSAS GARDUZA,GURROLA ROMERO KARLA,0.217391
1,delegación SEDESOL en México,739388,SECRETARIA DE DESARROLLO SOCIAL DELEGACION EST...,"DESARROLLO DE NEGOCIOS DEL GOLFO, S.A. DE CV.",0.256410
2,delegación SEDESOL en Tabasco,1072,SECRETARIA DE DESARROLLO SOCIAL DELEGACION TAB...,"CONSTRUCTORA Y DESARROLLADORA UNIDA JOMAR, S.A...",0.260870
3,delegación SEDESOL en Tabasco,109620,SERVISEG S.A. DE C.V.,"DAJIDE, S.A. DE CV.",0.250000
4,delegación SEDESOL en Tabasco,1109.9,COMERCIALIZADORA COMPUTEL DEL SURESTE S.A. DE...,"COMERCIALIZADORA COMAPELE, S.A. DE CV.",0.171429
